## Welcome to the quick notebook on using Llama 2 🦙 🚢

### To begin, make sure you:
1. pip install weaviate-client
2. pip install llama-index
3. pip install replicate 

### You will also need to have an access key for:
1. [OpenAI](https://openai.com/)
2. [Replicate](https://replicate.com/)

In [ ]:
%pip install  -U weaviate-client llama-index llama-index-vector-stores-weaviate llama-index-llms-replicate replicate 

## Connect to Weaviate

In [ ]:
import os
import weaviate

client = weaviate.connect_to_embedded(
    headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"]
    }
)

## Create Schema

In [2]:
from weaviate import classes as wvc
# clean slate
client.collections.delete("BlogPost")

collection = client.collections.create(
    name="BlogPost",
    description="Blog post from the Weaviate website.",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-small"
    ),
    properties=[
        wvc.config.Property(
            name="text", description="Content from the blog post", data_type=wvc.config.DataType.TEXT
        )
    ]
)

print("Collection was created.")

Collection was created.


{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/blogpost/EOnZvpRaS761/proplengths does not exist, creating new tracker","time":"2024-12-17T17:22:21-03:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-12-17T17:22:21-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"Created shard blogpost_EOnZvpRaS761 in 1.662125ms","time":"2024-12-17T17:22:21-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":

## Load data

In [3]:
from llama_index.core import SimpleDirectoryReader

# load the blogs in using the reader
blogs = SimpleDirectoryReader('./data').load_data()

## Construct vector store

In [4]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext

vector_store = WeaviateVectorStore(weaviate_client=client, index_name="BlogPost")

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(blogs, storage_context=storage_context)

# Let's define our Replicate API KEY

In [5]:
os.environ["REPLICATE_API_TOKEN"] = "<YOUR REPLICATE API KEY HERE>"

## Query time 

In [6]:
query_engine = index.as_query_engine()

In [7]:
response = query_engine.query("What is ref2vec? How does this feature help with building recommendation systems?")
print(response)

Ref2Vec is a module that allows for the vectorization of a data object based on its cross-references to other objects. It derives a centroid vector from the cross-referenced vectors to represent the referencing object. This approach enables the characterization of the referencing object through its relationships and actions, which can be refined over time. In building recommendation systems, Ref2Vec helps by representing a user's interests through a graph of cross-references from the user to objects they have interacted with. This unique representation of each user's preferences can then be used to rank search results based on relevance to the user's interests, enhancing the quality of recommendations provided.


In [8]:
response = query_engine.query("What is new in Weaviate 1.28?")
print(response)

Role-based access control (RBAC), enhanced async indexing improvements, native Japanese language support, conflict resolution improvements, new Japanese `kagome_ja` tokenizer for keyword/hybrid search, groundwork for Keyword & Hybrid search improvements with BlockMax WAND, Voyage AI Multimodal model support, and Weaviate Embeddings are new in Weaviate 1.28.
